<a href="https://colab.research.google.com/github/rajdeepbanerjee-git/JNCLectures_Intro_to_ML/blob/main/Week11/NN_comp_graph_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

In [ ]:
x = np.array([ [0, 1], [1, 0], [1, 1],[0, 0] ]).T
y = np.array([[1], [1], [0], [0]]).T
print(x.shape,y.shape)

(2, 4) (1, 4)


In [ ]:
num_input = 2
num_hidden = 5
num_output = 1

In [ ]:
W1_tf = tf.Variable(initial_value = tf.random.uniform(shape = (num_input, num_hidden)))
W2_tf = tf.Variable(initial_value = tf.random.uniform(shape = (num_hidden, num_output)))

In [ ]:
def model(inputs):
  return tf.sigmoid(tf.matmul(tf.sigmoid(tf.matmul(inputs, W1_tf)), W2_tf))

In [ ]:
x = x.astype(np.float32)
y = y.astype(np.float32)

In [ ]:
def square_loss(targets, predictions):
  per_sample_losses = tf.square(targets - predictions)
  return tf.reduce_mean(per_sample_losses)

In [ ]:
# prtevious metrod for training step

learning_rate = 0.01

def training_step(inputs, targets):
  with tf.GradientTape(persistent=True) as tape:
    predictions = model(inputs)
    loss = square_loss(predictions, targets)
  grad_loss_wrt_W1 = tape.gradient(loss, W1_tf)
  grad_loss_wrt_W2 = tape.gradient(loss, W2_tf)
  W1_tf.assign_sub(grad_loss_wrt_W1 * learning_rate)
  W2_tf.assign_sub(grad_loss_wrt_W2 * learning_rate)

  return loss


In [52]:
# breakdown to see derivatives
def training_step(inputs, targets):
  with tf.GradientTape(persistent=True) as tape:
      z1_ = tf.matmul(inputs, W1_tf)
      h1_ = tf.sigmoid(z1_)
      z2_ = tf.matmul(h1_, W2_tf)
      predictions = tf.sigmoid(z2_)
      loss = square_loss(predictions, targets)

  # calculate individual gradients in the graph
  dL_dh2 = tape.gradient(loss, predictions)
  dh2_dz2 = tape.gradient(predictions, z2_)
  dz2_dw2 = tape.gradient(z2_, W2_tf)
  dz2_dh1 = tape.gradient(z2_, h1_)
  dh1_dz1 = tape.gradient(h1_, z1_)
  dz1_dw1 = tape.gradient(z1_, W1_tf)
  derivatives = [dL_dh2, dh2_dz2, dz2_dw2, dz2_dh1, dh1_dz1, dz1_dw1]

  # calculate weight updates
  grad_loss_wrt_W1 = tape.gradient(loss, W1_tf)
  grad_loss_wrt_W2 = tape.gradient(loss, W2_tf)
  W1_tf.assign_sub(grad_loss_wrt_W1 * learning_rate)
  W2_tf.assign_sub(grad_loss_wrt_W2 * learning_rate)

  return derivatives, grad_loss_wrt_W2, grad_loss_wrt_W1


In [53]:
learning_rate = 0.01
num_iterations = 2
for step in range(num_iterations):
  print(training_step(inputs = x.T, targets = y))


([<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.09918898],
       [0.0973295 ],
       [0.10999888],
       [0.08538416]], dtype=float32)>, <tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.21064618],
       [0.21210788],
       [0.20160098],
       [0.22083817]], dtype=float32)>, <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[2.4191177],
       [2.7733495],
       [2.4692047],
       [2.355501 ],
       [2.286208 ]], dtype=float32)>, <tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.07431445, -0.00742008,  0.23206021,  0.21569633,  0.9086732 ],
       [ 0.07431445, -0.00742008,  0.23206021,  0.21569633,  0.9086732 ],
       [ 0.07431445, -0.00742008,  0.23206021,  0.21569633,  0.9086732 ],
       [ 0.07431445, -0.00742008,  0.23206021,  0.21569633,  0.9086732 ]],
      dtype=float32)>, <tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.2087895 , 0.20568816, 0.24515685, 0.2267834 , 0.24482241],
       [0.2499487 , 0.20707038, 0.22094807, 0.2

#### Assignment:
Use the individual gradients found using gradient tape to update W1 and W2 instead of directly using gradient of loss wrt W1 and W2.